Tensor，又名张量，读者可能对这个名词似曾相识，因它不仅在PyTorch中出现过，它也是Theano、TensorFlow、 Torch和MxNet中重要的数据结构。关于张量的本质不乏深度的剖析，但从工程角度来讲，可简单地认为它就是一个数组，且支持高效的科学计算。它可以是一个数（标量）、一维数组（向量）、二维数组（矩阵）和更高维的数组（高阶数据）。Tensor和Numpy的ndarrays类似，但PyTorch的tensor支持GPU加速。

In [1]:
from __future__ import print_function
import torch as t
t.__version__

'0.3.1'

###  3.1.1 基础操作

学习过Numpy的读者会对本节内容感到非常熟悉，因tensor的接口有意设计成与Numpy类似，以方便用户使用。但不熟悉Numpy也没关系，本节内容并不要求先掌握Numpy。

从接口的角度来讲，对tensor的操作可分为两类：

1. `torch.function`，如`torch.save`等。
2. 另一类是`tensor.function`，如`tensor.view`等。

为方便使用，对tensor的大部分操作同时支持这两类接口，在本书中不做具体区分，如`torch.sum (torch.sum(a, b))`与`tensor.sum (a.sum(b))`功能等价。

而从存储的角度来讲，对tensor的操作又可分为两类：

1. 不会修改自身的数据，如 `a.add(b)`， 加法的结果会返回一个新的tensor。
2. 会修改自身的数据，如 `a.add_(b)`， 加法的结果仍存储在a中，a被修改了。

函数名以`_`结尾的都是inplace方式, 即会修改调用者自己的数据，在实际应用中需加以区分。

#### 创建Tensor

在PyTorch中新建tensor的方法有很多，具体如表3-1所示。

表3-1: 常见新建tensor的方法

|函数|功能|
|:---:|:---:|
|Tensor(\*sizes)|基础构造函数|
|ones(\*sizes)|全1Tensor|
|zeros(\*sizes)|全0Tensor|
|eye(\*sizes)|对角线为1，其他为0|
|arange(s,e,step|从s到e，步长为step|
|linspace(s,e,steps)|从s到e，均匀切分成steps份|
|rand/randn(\*sizes)|均匀/标准分布|
|normal(mean,std)/uniform(from,to)|正态分布/均匀分布|
|randperm(m)|随机排列|

其中使用`Tensor`函数新建tensor是最复杂多变的方式，它既可以接收一个list，并根据list的数据新建tensor，也能根据指定的形状新建tensor，还能传入其他的tensor，下面举几个例子。

In [3]:
t.Tensor(5,3)


 0.0000e+00  0.0000e+00  3.0824e-29
 2.5244e-29  5.6052e-45  1.6115e-43
 1.5554e-43  1.5975e-43  5.6052e-44
 7.4269e-44  6.1657e-44  7.1466e-44
 5.7453e-44  1.3400e-08  2.1029e+37
[torch.FloatTensor of size 5x3]

In [6]:
x = t.Tensor([[1,2],[3,4]])

In [5]:
 t.Tensor([[1,2],[3,4]]).tolist()

[[1.0, 2.0], [3.0, 4.0]]

In [9]:
x.size()

torch.Size([2, 2])

In [10]:
x


 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [12]:
x.numel()

4

In [13]:
x.numel()

4

In [14]:
# 除了tensor.size()，还可以利用tensor.shape直接查看tensor的形状，tensor.shape等价于tensor.size()

In [15]:
x.shape

torch.Size([2, 2])

In [17]:
x.size()

torch.Size([2, 2])

In [18]:
x.shape??

需要注意的是，t.Tensor(*sizes)创建tensor时，系统不会马上分配空间，只是会计算剩余的内存是否足够使用，使用到tensor时才会分配，而其它操作都是在创建完tensor之后马上进行空间分配。其它常用的创建tensor的方法举例如下。

In [19]:
t.ones(2,3)


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

通过tensor.view方法可以调整tensor的形状，但必须保证调整前后元素总数一致。view不会修改自身的数据，返回的新tensor与源tensor共享内存，也即更改其中的一个，另外一个也会跟着改变。在实际应用中可能经常需要添加或减少某一维度，这时候squeeze和unsqueeze两个函数就派上用场了。

In [23]:
a = t.arange(0,6)
a


 0
 1
 2
 3
 4
 5
[torch.FloatTensor of size 6]

In [24]:
a.view(2,3)


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]

In [25]:
b = a.view(-1, 3) # 当某一维为-1的时候，会自动计算它的大小
b


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]

In [26]:
b.size()

torch.Size([2, 3])

In [28]:
ss = b.unsqueeze(1) # 注意形状，在第1维（下标从0开始）上增加“１”

In [29]:
ss


(0 ,.,.) = 
  0  1  2

(1 ,.,.) = 
  3  4  5
[torch.FloatTensor of size 2x1x3]

In [30]:
ss.size()

torch.Size([2, 1, 3])

In [31]:
b.unsqueeze(-2)


(0 ,.,.) = 
  0  1  2

(1 ,.,.) = 
  3  4  5
[torch.FloatTensor of size 2x1x3]

In [33]:
b


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]

In [34]:
b.view(1,1,1,2,3)


(0 ,0 ,0 ,.,.) = 
  0  1  2
  3  4  5
[torch.FloatTensor of size 1x1x1x2x3]

In [40]:
c = b.view(1,1,1,2,3)
c.squeeze(0) # 压缩第0维的“１”


(0 ,0 ,.,.) = 
  0  1  2
  3  4  5
[torch.FloatTensor of size 1x1x2x3]

In [41]:
c.squeeze()# 把所有维度为“1”的压缩


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]

In [44]:
a[1] = 100
b  # a修改，b作为view之后的，也会跟着修改


   0  100    2
   3    4    5
[torch.FloatTensor of size 2x3]

resize是另一种可用来调整size的方法，但与view不同，它可以修改tensor的大小。如果新大小超过了原大小，会自动分配新的内存空间，而如果新大小小于原大小，则之前的数据依旧会被保存，看一个例子。

In [45]:
b.resize_(1,3)


   0  100    2
[torch.FloatTensor of size 1x3]

In [46]:
b


   0  100    2
[torch.FloatTensor of size 1x3]

In [47]:
b.resize_(3, 3) # 旧的数据依旧保存着，多出的大小会分配新空间
b


   0.0000  100.0000    2.0000
   3.0000    4.0000    5.0000
   0.0000    0.0000    0.0000
[torch.FloatTensor of size 3x3]

索引操作
Tensor支持与numpy.ndarray类似的索引操作，语法上也类似，下面通过一些例子，讲解常用的索引操作。如无特殊说明，索引出来的结果与原tensor共享内存，也即修改一个，另一个会跟着修改。

In [48]:
a = t.randn(3,4)
a


-1.0263 -0.2313 -0.5471  0.7467
-2.9301 -0.0249  0.6750 -0.7959
 0.0932  1.5405 -0.2693 -0.4260
[torch.FloatTensor of size 3x4]

In [49]:
a[0]


-1.0263
-0.2313
-0.5471
 0.7467
[torch.FloatTensor of size 4]

In [50]:
a[:2]


-1.0263 -0.2313 -0.5471  0.7467
-2.9301 -0.0249  0.6750 -0.7959
[torch.FloatTensor of size 2x4]

In [51]:
print(a[0:1, :2]) # 第0行，前两列 
print(a[0, :2]) # 注意两者的区别：形状不同


-1.0263 -0.2313
[torch.FloatTensor of size 1x2]


-1.0263
-0.2313
[torch.FloatTensor of size 2]



In [52]:
a < 0


 1  1  1  0
 1  1  0  1
 0  0  1  1
[torch.ByteTensor of size 3x4]

In [54]:
a[a>1] # 等价于a.masked_select(a>1)
# 选择结果与原tensor不共享内存空间


 1.5405
[torch.FloatTensor of size 1]

In [56]:
a


-1.0263 -0.2313 -0.5471  0.7467
-2.9301 -0.0249  0.6750 -0.7959
 0.0932  1.5405 -0.2693 -0.4260
[torch.FloatTensor of size 3x4]

In [55]:
a[t.LongTensor([0,1])] # 第0行和第1行


-1.0263 -0.2313 -0.5471  0.7467
-2.9301 -0.0249  0.6750 -0.7959
[torch.FloatTensor of size 2x4]

In [57]:
t.LongTensor([0,1])


 0
 1
[torch.LongTensor of size 2]

In [58]:
t.Tensor([0,1])


 0
 1
[torch.FloatTensor of size 2]

其它常用的选择函数如表3-2所示。

表3-2常用的选择函数

函数|功能|
:---:|:---:|
index_select(input, dim, index)|在指定维度dim上选取，比如选取某些行、某些列
masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor进行选取
non_zero(input)|非0元素的下标
gather(input, dim, index)|根据index，在dim维度上选取数据，输出的size与index一样


`gather`是一个比较复杂的操作，对一个2维tensor，输出的每个元素如下：

```python
out[i][j] = input[index[i][j]][j]  # dim=0
out[i][j] = input[i][index[i][j]]  # dim=1
```
三维tensor的`gather`操作同理，下面举几个例子。

参数: 
input (Tensor) – 源tensor 
dim (int) – 指定的轴数（维数） 
index (LongTensor) – 需要聚集起来的数据的索引 
out (Tensor, optional) – 目标tensor

简单来说，就是在Tensor(input)的众多维度中针对某一维度(dim参数)，使用一维Tensor（index）进行索引，并对其他维度进行遍历。

In [59]:
a = t.arange(0,16).view(4,4)
a


  0   1   2   3
  4   5   6   7
  8   9  10  11
 12  13  14  15
[torch.FloatTensor of size 4x4]

In [81]:
an = t.LongTensor([[0,1,2,3]])

In [82]:
an


 0  1  2  3
[torch.LongTensor of size 1x4]

In [92]:
# 选取对角线的元素
index = t.LongTensor([[0,1,2,3]])
a.gather(0,index)


  0   5  10  15
[torch.FloatTensor of size 1x4]

In [64]:
# 选取反对角线上的元素
index = t.LongTensor([[3,2,1,0]]).t()
a.gather(1, index)


  3
  6
  9
 12
[torch.FloatTensor of size 4x1]

In [94]:
# 选取反对角线上的元素，注意与上面的不同
index = t.LongTensor([[3,2,1,0]])
a.gather(0, index)


 12   9   6   3
[torch.FloatTensor of size 1x4]

In [95]:
# 选取两个对角线上的元素
index = t.LongTensor([[0,1,2,3],[3,2,1,0]]).t()
b = a.gather(1, index)
b


  0   3
  5   6
 10   9
 15  12
[torch.FloatTensor of size 4x2]

与`gather`相对应的逆操作是`scatter_`，`gather`把数据从input中按index取出，而`scatter_`是把取出的数据再放回去。注意`scatter_`函数是inplace操作。

```python
out = input.gather(dim, index)
-->近似逆操作
out = Tensor()
out.scatter_(dim, index)
```

In [96]:
# 把两个对角线元素放回去到指定位置
c = t.zeros(4,4)
c.scatter_(1, index, b)


  0   0   0   3
  0   5   6   0
  0   9  10   0
 12   0   0  15
[torch.FloatTensor of size 4x4]

In [97]:
x = t.arange(0,27).view(3,3,3)
x


(0 ,.,.) = 
   0   1   2
   3   4   5
   6   7   8

(1 ,.,.) = 
   9  10  11
  12  13  14
  15  16  17

(2 ,.,.) = 
  18  19  20
  21  22  23
  24  25  26
[torch.FloatTensor of size 3x3x3]

In [85]:
x[[1, 2], [1, 2], [2, 0]] # x[1,1,2]和x[2,2,0]


 14
 24
[torch.FloatTensor of size 2]

In [87]:
x[[1, 2],[1,2]] 


 12  13  14
 24  25  26
[torch.FloatTensor of size 2x3]

In [88]:
x[[2, 1, 0], [0], [1]] # x[2,0,1],x[1,0,1],x[0,0,1]


 19
 10
  1
[torch.FloatTensor of size 3]

In [89]:
x[[0, 2], ...] # x[0] 和 x[2]


(0 ,.,.) = 
   0   1   2
   3   4   5
   6   7   8

(1 ,.,.) = 
  18  19  20
  21  22  23
  24  25  26
[torch.FloatTensor of size 2x3x3]

In [103]:
s = t.arange(0,16).view(4,4)

In [104]:
s


  0   1   2   3
  4   5   6   7
  8   9  10  11
 12  13  14  15
[torch.FloatTensor of size 4x4]

In [112]:
s[[1],...]


 4  5  6  7
[torch.FloatTensor of size 1x4]

#### Tensor类型

Tensor有不同的数据类型，如表3-3所示，每种类型分别对应有CPU和GPU版本(HalfTensor除外)。默认的tensor是FloatTensor，可通过`t.set_default_tensor_type` 来修改默认tensor类型(如果默认类型为GPU tensor，则所有操作都将在GPU上进行)。Tensor的类型对分析内存占用很有帮助。例如对于一个size为(1000, 1000, 1000)的FloatTensor，它有`1000*1000*1000=10^9`个元素，每个元素占32bit/8 = 4Byte内存，所以共占大约4GB内存/显存。HalfTensor是专门为GPU版本设计的，同样的元素个数，显存占用只有FloatTensor的一半，所以可以极大缓解GPU显存不足的问题，但由于HalfTensor所能表示的数值大小和精度有限[^2]，所以可能出现溢出等问题。

[^2]: https://stackoverflow.com/questions/872544/what-range-of-numbers-can-be-represented-in-a-16-32-and-64-bit-ieee-754-syste

表3-3: tensor数据类型

数据类型|	CPU tensor	|GPU tensor|
:---:|:---:|:--:|
32-bit 浮点|	torch.FloatTensor	|torch.cuda.FloatTensor
64-bit 浮点|	torch.DoubleTensor|	torch.cuda.DoubleTensor
16-bit 半精度浮点|	N/A	|torch.cuda.HalfTensor
8-bit 无符号整形(0~255)|	torch.ByteTensor|	torch.cuda.ByteTensor
8-bit 有符号整形(-128~127)|	torch.CharTensor	|torch.cuda.CharTensor
16-bit 有符号整形  |	torch.ShortTensor|	torch.cuda.ShortTensor
32-bit 有符号整形 	|torch.IntTensor	|torch.cuda.IntTensor
64-bit 有符号整形  	|torch.LongTensor	|torch.cuda.LongTensor

各数据类型之间可以互相转换，`type(new_type)`是通用的做法，同时还有`float`、`long`、`half`等快捷方法。CPU tensor与GPU tensor之间的互相转换通过`tensor.cuda`和`tensor.cpu`方法实现。Tensor还有一个`new`方法，用法与`t.Tensor`一样，会调用该tensor对应类型的构造函数，生成与当前tensor类型一致的tensor。

In [113]:
# 设置默认tensor，注意参数是字符串
t.set_default_tensor_type('torch.IntTensor')

In [114]:
a = t.Tensor(2,3)
a


 0.0000e+00  0.0000e+00  2.7843e+08
 2.6844e+08  2.0000e+00  0.0000e+00
[torch.IntTensor of size 2x3]

In [115]:
a.float()


 0.0000e+00  0.0000e+00  2.7843e+08
 2.6844e+08  2.0000e+00  0.0000e+00
[torch.FloatTensor of size 2x3]

In [116]:
a.type_as(b)


 0.0000e+00  0.0000e+00  2.7843e+08
 2.6844e+08  2.0000e+00  0.0000e+00
[torch.FloatTensor of size 2x3]

In [119]:
d = a.new(2,3)  # 等价于torch.IntTensor(2,3)
d


 0.0000e+00  0.0000e+00  2.7023e+08
 2.6844e+08  2.7020e+08  0.0000e+00
[torch.IntTensor of size 2x3]

In [120]:
a.new??

In [121]:
# 恢复之前的默认设置
t.set_default_tensor_type('torch.FloatTensor')

#### 逐元素操作

这部分操作会对tensor的每一个元素(point-wise，又名element-wise)进行操作，此类操作的输入与输出形状一致。常用的操作如表3-4所示。

表3-4: 常见的逐元素操作

|函数|功能|
|:--:|:--:|
|abs/sqrt/div/exp/fmod/log/pow..|绝对值/平方根/除法/指数/求余/求幂..|
|cos/sin/asin/atan2/cosh..|相关三角函数|
|ceil/round/floor/trunc| 上取整/四舍五入/下取整/只保留整数部分|
|clamp(input, min, max)|超过min和max部分截断|
|sigmod/tanh..|激活函数

对于很多操作，例如div、mul、pow、fmod等，PyTorch都实现了运算符重载，所以可以直接使用运算符。如`a ** 2` 等价于`torch.pow(a,2)`, `a * 2`等价于`torch.mul(a,2)`。

其中`clamp(x, min, max)`的输出满足以下公式：
$$
y_i =
\begin{cases}
min,  & \text{if  } x_i \lt min \\
x_i,  & \text{if  } min \le x_i \le max  \\
max,  & \text{if  } x_i \gt max\\
\end{cases}
$$
`clamp`常用在某些需要比较大小的地方，如取一个tensor的每个元素与另一个数的较大值。

In [122]:
a = t.arange(0,6).view(2,3)
a


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]

In [123]:
t.cos(a)


 1.0000  0.5403 -0.4161
-0.9900 -0.6536  0.2837
[torch.FloatTensor of size 2x3]

In [124]:
# 取a中的每一个元素与3相比较大的一个 (小于3的截断成3)
print(a)
t.clamp(a, min=3)


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]




 3  3  3
 3  4  5
[torch.FloatTensor of size 2x3]

####  归并操作 
此类操作会使输出形状小于输入形状，并可以沿着某一维度进行指定操作。如加法`sum`，既可以计算整个tensor的和，也可以计算tensor中每一行或每一列的和。常用的归并操作如表3-5所示。

表3-5: 常用归并操作

|函数|功能|
|:---:|:---:|
|mean/sum/median/mode|均值/和/中位数/众数|
|norm/dist|范数/距离|
|std/var|标准差/方差|
|cumsum/cumprod|累加/累乘|

以上大多数函数都有一个参数**`dim`**，用来指定这些操作是在哪个维度上执行的。关于dim(对应于Numpy中的axis)的解释众说纷纭，这里提供一个简单的记忆方式：

假设输入的形状是(m, n, k)

- 如果指定dim=0，输出的形状就是(1, n, k)或者(n, k)
- 如果指定dim=1，输出的形状就是(m, 1, k)或者(m, k)
- 如果指定dim=2，输出的形状就是(m, n, 1)或者(m, n)

size中是否有"1"，取决于参数`keepdim`，`keepdim=True`会保留维度`1`。注意，以上只是经验总结，并非所有函数都符合这种形状变化方式，如`cumsum`。

In [125]:
b = t.ones(2, 3)
b.sum(dim = 0, keepdim=True)


 2  2  2
[torch.FloatTensor of size 1x3]

In [126]:
b


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [127]:
b.sum(dim = 0, keepdim=False)


 2
 2
 2
[torch.FloatTensor of size 3]

In [128]:
b.sum(dim=1)


 3
 3
[torch.FloatTensor of size 2]

In [134]:
b


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [135]:
a = t.arange(0, 6).view(2, 3)
print(a)
a.cumsum(dim=1) # 沿着行累加


 0  1  2
 3  4  5
[torch.FloatTensor of size 2x3]




  0   1   3
  3   7  12
[torch.FloatTensor of size 2x3]

#### 比较
比较函数中有一些是逐元素比较，操作类似于逐元素操作，还有一些则类似于归并操作。常用比较函数如表3-6所示。

表3-6: 常用比较函数

|函数|功能|
|:--:|:--:|
|gt/lt/ge/le/eq/ne|大于/小于/大于等于/小于等于/等于/不等|
|topk|最大的k个数|
|sort|排序|
|max/min|比较两个tensor最大最小值|

表中第一行的比较操作已经实现了运算符重载，因此可以使用`a>=b`、`a>b`、`a!=b`、`a==b`，其返回结果是一个`ByteTensor`，可用来选取元素。max/min这两个操作比较特殊，以max来说，它有以下三种使用情况：
- t.max(tensor)：返回tensor中最大的一个数
- t.max(tensor,dim)：指定维上最大的数，返回tensor和下标
- t.max(tensor1, tensor2): 比较两个tensor相比较大的元素

至于比较一个tensor和一个数，可以使用clamp函数。下面举例说明。

In [136]:
a = t.linspace(0, 15, 6).view(2, 3)
a


  0   3   6
  9  12  15
[torch.FloatTensor of size 2x3]

In [137]:
b = t.linspace(15, 0, 6).view(2, 3)
b


 15  12   9
  6   3   0
[torch.FloatTensor of size 2x3]

In [138]:
a > b


 0  0  0
 1  1  1
[torch.ByteTensor of size 2x3]

In [139]:
a[a>b]


  9
 12
 15
[torch.FloatTensor of size 3]

In [140]:
t.max(a)

15.0

In [146]:
a


  0   3   6
  9  12  15
[torch.FloatTensor of size 2x3]

In [147]:
t.max(a, dim=1) 
# 第一个返回值的15和6分别表示第0行和第1行最大的元素
# 第二个返回值的0和0表示上述最大的数是该行第0个元素

(
   6
  15
 [torch.FloatTensor of size 2], 
  2
  2
 [torch.LongTensor of size 2])

In [148]:
# 比较a和10较大的元素
t.clamp(a, min=10)


 10  10  10
 10  12  15
[torch.FloatTensor of size 2x3]

#### 线性代数

PyTorch的线性函数主要封装了Blas和Lapack，其用法和接口都与之类似。常用的线性代数函数如表3-7所示。

表3-7: 常用的线性代数函数

|函数|功能|
|:---:|:---:|
|trace|对角线元素之和(矩阵的迹)|
|diag|对角线元素|
|triu/tril|矩阵的上三角/下三角，可指定偏移量|
|mm/bmm|矩阵乘法，batch的矩阵乘法|
|addmm/addbmm/addmv/addr/badbmm..|矩阵运算
|t|转置|
|dot/cross|内积/外积
|inverse|求逆矩阵
|svd|奇异值分解

具体使用说明请参见官方文档[^3]，需要注意的是，矩阵的转置会导致存储空间不连续，需调用它的`.contiguous`方法将其转为连续。
[^3]: http://pytorch.org/docs/torch.html#blas-and-lapack-operations

In [149]:
b = a.t()
b.is_contiguous()

False

In [150]:
b


  0   9
  3  12
  6  15
[torch.FloatTensor of size 3x2]

In [151]:
b.contiguous()


  0   9
  3  12
  6  15
[torch.FloatTensor of size 3x2]

Tensor和Numpy数组之间具有很高的相似性，彼此之间的互操作也非常简单高效。需要注意的是，Numpy和Tensor共享内存。由于Numpy历史悠久，支持丰富的操作，所以当遇到Tensor不支持的操作时，可先转成Numpy数组，处理后再转回tensor，其转换开销很小。



In [152]:
import numpy as np

In [153]:
a = np.ones([2,3],dtype=np.float32)
a

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [156]:
b = t.from_numpy(a)
b


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [157]:
t.Tensor(a)


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [158]:
a[0,1] =100

In [159]:
b


   1  100    1
   1    1    1
[torch.FloatTensor of size 2x3]

In [160]:
c = b.numpy() # a, b, c三个对象共享内存
c

array([[  1., 100.,   1.],
       [  1.,   1.,   1.]], dtype=float32)

In [161]:
# 注意： 当numpy的数据类型和Tensor的类型不一样的时候，数据会被复制，不会共享内存。

In [162]:
a = np.ones([2, 3])
a # 注意和上面的a的区别（dtype不是float32）

array([[1., 1., 1.],
       [1., 1., 1.]])

In [163]:
b = t.Tensor(a) # FloatTensor(double64或者float64)
b


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [164]:
c = t.from_numpy(a) # 注意c的类型（DoubleTensor）
c


 1  1  1
 1  1  1
[torch.DoubleTensor of size 2x3]

广播法则(broadcast)是科学运算中经常使用的一个技巧，它在快速执行向量化的同时不会占用额外的内存/显存。
Numpy的广播法则定义如下：

- 让所有输入数组都向其中shape最长的数组看齐，shape中不足的部分通过在前面加1补齐
- 两个数组要么在某一个维度的长度一致，要么其中一个为1，否则不能计算 
- 当输入数组的某个维度的长度为1时，计算时沿此维度复制扩充成一样的形状

PyTorch当前已经支持了自动广播法则，但是笔者还是建议读者通过以下两个函数的组合手动实现广播法则，这样更直观，更不易出错：

- `unsqueeze`或者`view`：为数据某一维的形状补1，实现法则1
- `expand`或者`expand_as`，重复数组，实现法则3；该操作不会复制数组，所以不会占用额外的空间。

注意，repeat实现与expand相类似的功能，但是repeat会把相同数据复制多份，因此会占用额外的空间。